In [1]:
import os
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
print(notebook_dir)
os.chdir(notebook_dir)

/home/jetson/cmap/athirdmapper


In [6]:
from utils import CLIP
import os
import pickle
os.chdir('../')
clip = CLIP('ViT-L-14-quickgelu')
text_features = {}
text_list = ['boxes', 'fire extinguisher', 'luggage', 'a single traffic cone', 'pile of traffic cones', 'trash bin', 'umbrella', 'trolley', 'chair', 'folded chair']
text_features['list'] = text_list
for text in text_list:
    text_features[text] = clip.encode_text([text])
os.chdir('./athirdmapper')
with open('text_features.pkl', 'wb') as f:
    pickle.dump(text_features, f)
print(text_list)

['boxes', 'fire extinguisher', 'luggage', 'a single traffic cone', 'pile of traffic cones', 'trash bin', 'umbrella', 'trolley', 'chair', 'folded chair']


In [1]:
import os
import pickle
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import cv2
def similarity(image_features, text_features):
    if torch.cuda.is_available(): 
        return image_features @ text_features.cpu().numpy().T
    return image_features @ text_features.numpy().T
def show_candidates(ind):
    for i, img_ind in enumerate(ind):
        r = len(ind)//9+1
        plt.figure('candidates', figsize=(12, r*2))
        plt.subplot(r, 9, i+1)
        img = cv2.imread('n_images/' + str(img_ind) + '.png')
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.tight_layout()
with open('features.pkl', 'rb') as file:
    features = pickle.load(file)
with open('features_ind.pkl', 'rb') as file:
    features_ind = pickle.load(file)
with open('text_features.pkl', 'rb') as file:
    text_features = pickle.load(file)
print(text_features['list'])

['umbrella', 'fire extinguisher', 'a single traffic cone', 'luggage', 'rc car', 'pile of traffic cone']


In [9]:
text_feature = text_features['umbrella']
similarities = similarity(features, text_feature).squeeze()*100
# get softmax
similarities_s = F.softmax(torch.tensor(similarities), dim=0).numpy()
similarities_s
M_s = np.max(similarities_s)
m_s = np.min(similarities_s)
nsim = np.zeros_like(similarities_s)
nsim[similarities_s > 0.01] = 1
m = int(np.sum(nsim))
# nsim[similarities_s < 0.01] = 0
# n = int(np.sum(nsim))
print(M_s, m_s, m)

sim_sort_ind = np.argsort(similarities, axis=0)[::-1]
sim_sort_ind = sim_sort_ind[:m]
show_candidates(sim_sort_ind)

0.04180763447116345 1.0038126478700598e-08 33


In [11]:
# print(sim_sort_ind)
# print(similarities_s[sim_sort_ind])
# get unique points
conf = {}
for index in sim_sort_ind:
    # print(len(features_ind[index]), features_ind[index])
    similarities_s = similarities[index]
    n_point = len(features_ind[index])
    for point in features_ind[index]:
        [s, n] = conf.get(tuple(point), [0,0])
        conf[tuple(point)] = [(s * n + similarities_s) / (n + 1), n + 1]
# sort confidence by value
conf_score = dict(sorted(conf.items(), key=lambda item: item[1], reverse=True))
conf_freq = dict(sorted(conf.items(), key=lambda item: item[1][1], reverse=True))

k = list(conf_score.keys())
v = list(conf_score.values())
print(k)
print(v)

K = list(conf_freq.keys())
V = list(conf_freq.values())
print(K)
print(V)

[(-5.25, -1.0, 0.0), (-5.25, -1.25, 0.0), (-5.0, -1.25, 0.0), (-5.0, -1.5, 0.0), (-4.75, -1.5, 0.0), (-5.0, -1.0, 0.0), (-5.5, -0.25, 0.0), (-5.5, -0.5, 0.0), (-5.25, -0.25, 0.0), (-4.5, -0.25, 0.0), (-5.0, -0.25, 0.0), (-5.25, -0.5, 0.0), (-4.75, -0.25, 0.0), (-4.25, -0.25, 0.0), (-4.75, -0.5, 0.0), (-4.25, -0.5, 0.0), (-5.5, 0.0, 0.0), (-5.5, -0.75, 0.0), (-5.25, -0.75, 0.0), (-5.0, -0.5, 0.0), (-4.75, -1.25, 0.0), (-4.5, -0.5, 0.0), (-5.5, 0.25, 0.0)]
[[11.755626587643064, 2], [11.5785278127305, 4], [11.339637713495614, 11], [11.326023906121918, 3], [11.25862031210545, 2], [11.222557687632388, 5], [11.217042897223592, 10], [11.155957641365077, 12], [11.149654017040689, 4], [11.085183770977892, 25], [11.08345574942853, 22], [11.081106124359241, 16], [11.074479674328002, 26], [11.05037466146257, 3], [10.981079755006817, 2], [10.96030921084008, 1], [10.910562755605326, 5], [10.90654840227406, 3], [10.905029436388247, 4], [10.858411146903839, 7], [10.828163860639961, 1], [10.80130332052